# Ukraine Refugee Mapping

In [ ]:
import json
import time
import requests

# google libraries
import googlemaps
import polyline

# mapping and shape utils
import folium
from folium import plugins

# data processing
import pandas as pd
import geopandas as gpd

## Get Border Crossings

In [ ]:
crossings_df = pd.read_csv('inputs/Ukraine_BorderCrossings.csv')

In [ ]:
crossings_df.rename(columns={"Lat":"lat", "Long":"lon", "Name - English":"name"},inplace=True)

In [ ]:
crossings_df = crossings_df[['name','Country','lat','lon']].dropna()
print(crossings_df.Country.unique())

# select only target countries, not Belarus or Russia
crossings_df = crossings_df[crossings_df['Country'].str.contains('Poland|Moldova|Romania|Slovakia|Hungary')]
print(crossings_df.Country.unique())

First, you need to enable the Google Directions API.

In [ ]:
gmaps = googlemaps.Client(key='YOUR_KEY_GOES_HERE')

Add custom basemaps to folium

In [ ]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

## Manual Routes KML

In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
manual_routes = gpd.read_file('inputs/QGIS_Ukraine_Routes.kml', driver='KML')

## Read in Locations

In [ ]:
df = pd.read_csv('ukraine_locations.csv')
df.head()

In [ ]:
conflicts = df[df['location_type']=='conflict_zone']

In [ ]:
conflicts.shape

In [ ]:
def get_closest(loc_lat, loc_lon, targets):
    chunk_size = 25
    list_targets = [targets[i:i+chunk_size] for i in range(0,targets.shape[0],chunk_size)]
    output = None
    closest_seconds = 100000000000
    closest_loc = None
    for i in list_targets:
        results = gmaps.distance_matrix(origins=[(loc_lat, loc_lon)],
                                        destinations=list(tuple(zip(i.lat, i.lon))), mode='transit')
        for idx, val in enumerate(results['rows'][0]['elements']):
            try:
                seconds = val['duration']['value']
            except Exception as e:
                continue
            if seconds <= closest_seconds:
                closest_seconds = seconds
                closest_loc = i.iloc[idx]
                output = val
    return closest_loc, output

In [ ]:
conflict_exit_routes = {}

In [ ]:
for kk, conflict in conflicts.iterrows():
    if conflict['#name'] not in conflict_exit_routes:
        print(conflict['#name'])
        closest_crossing, crossing_val = get_closest(conflict.latitude, conflict.longitude, crossings_df)
        if isinstance(closest_crossing, type(None)):
            print(f"{conflict['#name']} No routes found")
        conflict_exit_routes[conflict['#name']] = dict(crossing=closest_crossing, 
                                                       crossing_v=crossing_val)

In [ ]:
crossings_unique = set()
for kk, vv in conflict_exit_routes.items():
    try:
        crossings_unique.add(vv['crossing']['name'])
    except:
        continue

In [ ]:
print(crossings_unique)

In [ ]:
for kk, vv in conflict_exit_routes.items():
    if not isinstance(vv['crossing'], type(None)):
        vv['crossing'] = dict(vv['crossing'])
#         vv['reception'] = dict(vv['reception'])

In [ ]:
with open('ukraine_exit_routes_transit.json','w') as f:
    f.write(json.dumps(conflict_exit_routes))

In [ ]:
with open('ukraine_exit_routes.json','r') as f:
    conflict_exit_routes = json.loads(f.read())

## Directions and Plotting

In [ ]:
all_directions = {}
for kk, conflict in conflicts.iterrows():
    conflict_name = conflict['#name']
    print(f"Getting directions for conflict: {conflict_name}")
    
    xing = conflict_exit_routes[conflict_name]['crossing']
    try:
        directions_result = gmaps.directions((conflict.latitude, conflict.longitude),
                                         (xing['lat'], xing['lon']),
                                         mode="transit")
    except:
        directions_result = None
    all_directions[conflict_name] = directions_result

In [ ]:
with open('ukraine_border_crossing_directions_transit.json','w') as f:
    f.write(json.dumps(all_directions))

In [ ]:
with open('ukraine_border_crossing_directions_transit.json','r') as f:
    all_directions_transit = json.loads(f.read())

In [ ]:
with open('ukraine_border_crossing_directions.json','r') as f:
    all_directions = json.loads(f.read())

## Nearest Camp Map

Min/max normalization for population to obtain an appropriate line strkoe

In [ ]:
c_desc = conflicts.population.describe()

In [ ]:
def bucket_population(population):
    if population <= c_desc['25%']:
        stroke = 2.5
    elif population <= c_desc['50%']:
        stroke = 5
    elif population <= c_desc['75%']:
        stroke = 7.5
    else:
        stroke = 10
    return stroke
        

In [ ]:
conflicts['stroke'] = conflicts['population'].apply(lambda x: bucket_population(x))

In [ ]:
# Create Map
map = folium.Map(location=[conflicts.latitude.mean(),conflicts.longitude.mean()], zoom_start=6)

# Plot conflict starting points
for kk, start in conflicts.iterrows():
    start_m = folium.Marker([start.latitude, start.longitude], popup=start['#name'], 
                            icon=folium.Icon(icon='glyphicon glyphicon-fire', color='darkred'))
    start_m.add_to(map)
    
# Plot ending locations
for kk, vv in crossings_df.iterrows():
    icon = 'glyphicon glyphicon-road'
    color = 'orange'
    popup_text = f"<b>Crossing Name: </b>{vv['name']}<br>"\
                 f"<b>Country: </b>{vv['Country']}"
    popup = folium.Popup(popup_text, max_width=300,min_width=150)
    xing = folium.Marker([vv.lat, vv.lon], popup=popup, 
                          icon=folium.Icon(icon=icon, color=color))
    xing.add_to(map)

# plot exit routes (driving)
fg_d = folium.FeatureGroup("Driving")
for kk, vv in all_directions.items():
    stroke = int(conflicts[conflicts['#name']==kk]['stroke'])
    population = "{:,}".format(int(conflicts[conflicts['#name']==kk]['population']))
    directions = all_directions[kk]
    if not isinstance(directions, type(None)):
        distance = directions[0]['legs'][0]['distance']['text']
        duration = directions[0]['legs'][0]['duration']['text']
        end_location = directions[0]['legs'][0]['end_address']
        tooltip = f"Travel between <b>{kk}</b> and <b>{end_location}</b> by car is <b>"\
                  f"{distance}</b> and takes <b>{duration}</b>.</br></br>"\
                  f"<b>{population}</b> people are effected by this conflict."
        polyline_ = polyline.decode(directions[0]['overview_polyline']['points'])
        polyline_m = folium.PolyLine(polyline_, color='#4A89F3', tooltip=tooltip, weight=stroke)
        polyline_m.add_to(fg_d)
        
# plot exit routes (transit)
fg_t = folium.FeatureGroup("Transit")
for kk, vv in all_directions_transit.items():
    stroke = int(conflicts[conflicts['#name']==kk]['stroke'])
    population = "{:,}".format(int(conflicts[conflicts['#name']==kk]['population']))
    directions = all_directions_transit[kk]
    if not isinstance(directions, type(None)):
        distance = directions[0]['legs'][0]['distance']['text']
        duration = directions[0]['legs'][0]['duration']['text']
        end_location = directions[0]['legs'][0]['end_address']
        tooltip = f"Travel between <b>{kk}</b> and <b>{end_location}</b> by transit is <b>"\
                  f"{distance}</b> and takes <b>{duration}</b>.</br></br>"\
                  f"<b>{population}</b> people are effected by this conflict."
        polyline_ = polyline.decode(directions[0]['overview_polyline']['points'])
        polyline_m = folium.PolyLine(polyline_, color='#7570b3', tooltip=tooltip, weight=stroke)
        polyline_m.add_to(fg_t)        
        
fg_d.add_to(map)
fg_t.add_to(map)
folium.Choropleth(
    manual_routes,
    line_weight=3,
    line_color='#f03b20',
    name='Manually generated routes'
).add_to(map)
        
# Add custom basemaps
basemaps['Google Satellite Hybrid'].add_to(map)
# basemaps['Esri Satellite'].add_to(map)
# basemaps['Google Satellite'].add_to(map)
basemaps['Google Maps'].add_to(map)

# Add a layer control panel to the map.
map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(map)

In [ ]:
def add_legend(map):
    legend_html = """
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@100;300;400&display=swap');
    </style>
    
     <div style="
     padding-left:5px; padding-top:5px;
     position: fixed; 
     bottom: 50px; left: 50px; width: 160px; height: 120px;   
     border:2px solid grey; z-index:9999; border-radius: 15px;
     
     background-color:white;
     opacity: .85;
     
     font-size:14px;
     font-weight: bold;
     font-family: 'Roboto', sans-serif;
     ">

     <div class="awesome-marker-icon-darkred awesome-marker" style="margin-top: 10px; margin-left:5px;">
         <i class="fa-rotate-0 glyphicon glyphicon-glyphicon glyphicon-fire icon-white"></i>
     </div>
     <div style="margin-left:40px; margin-top:20px">Conflict Area</div>
     
     <div class="awesome-marker-icon-orange awesome-marker" style="margin-top: 60px; margin-left:5px;">
         <i class="fa-rotate-0 glyphicon glyphicon-glyphicon glyphicon-road icon-white"></i>
     </div>
     <div style="margin-left:40px; margin-top:25px">Border Crossing</div>     
     
     
      </div> """.format( title = "Legend html")
    map.get_root().html.add_child(folium.Element( legend_html ))
    return map


# Add map title
# title_html = '''
#         <style>
#         @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@100;300;400&display=swap');
#         </style>
#         <h2 align="center" style="font-family: 'Roboto', sans-serif;">
#             <b>Ukraine Conflict and Refugee Camps</b>
#         </h2>
#         '''
# map.get_root().html.add_child(folium.Element(title_html))

In [ ]:
map = add_legend(map)

In [ ]:
display(map)

In [ ]:
# save map
map.save('Ukraine_Border_Crossing_Map_UPDATED.html')

In [ ]:
conflicts['#name'].unique()

## Generate Conflict Focus Plot

In [ ]:
def plot_conflict_focus(conflict_name, conflicts, crossings_df):
    # Create Map
    conflict = conflicts[conflicts['#name']==conflict_name]
    map = folium.Map(location=[conflict.latitude,conflict.longitude], zoom_start=6)

    # Plot conflict point
    start_m = folium.Marker([conflict.latitude, conflict.longitude], popup=conflict_name, 
                            icon=folium.Icon(icon='glyphicon glyphicon-fire', color='darkred'))
    start_m.add_to(map)

    for kk, vv in crossings_df.iterrows():
        icon = 'glyphicon glyphicon-road'
        color = 'orange'
        popup_text = f"<b>Crossing Name: </b>{vv['name']}<br>"\
                     f"<b>Country: </b>{vv['Country']}"
        popup = folium.Popup(popup_text, max_width=300,min_width=150)
        xing = folium.Marker([vv.lat, vv.lon], popup=popup, 
                              icon=folium.Icon(icon=icon, color=color))
        xing.add_to(map)

    # Plot all possible refugee movement EXCEPT CLOSEST
    for kk, directions in all_directions[conflict_name].items():
        try:
            # this is closest route so set color accordingly
            if closest[conflict_name] == kk:
                continue
            else:
                color = '#4A89F3'
            distance = directions[0]['legs'][0]['distance']['text']
            duration = directions[0]['legs'][0]['duration']['text']
            tooltip = f"Travel between <b>{conflict_name}</b> and <b>{kk}</b> by car is <b>"\
                      f"{distance}</b> and takes <b>{duration}</b>."
            polyline_ = polyline.decode(directions[0]['overview_polyline']['points'])
            polyline_m = folium.PolyLine(polyline_, color=color, tooltip=tooltip, weight=5)
            polyline_m.add_to(map)
        except Exception as e:
            print(e)
            print(f"No directions between: {conflict_name} and {kk}")
    
    # Plot closest ON TOP
    for kk, directions in all_directions[conflict_name].items():
        try:
            # this is closest route so set color accordingly
            if closest[conflict_name] == kk:
                color = 'yellow'
            else:
                continue
            distance = directions[0]['legs'][0]['distance']['text']
            duration = directions[0]['legs'][0]['duration']['text']
            tooltip = f"This is the predicted route. Travel between <b>{conflict_name}</b> and <b>{kk}</b> by car is <b>"\
                      f"{distance}</b> and takes <b>{duration}</b>."
            polyline_ = polyline.decode(directions[0]['overview_polyline']['points'])
            polyline_m = folium.PolyLine(polyline_, color=color, tooltip=tooltip, weight=15)
            polyline_m.add_to(map)
        except Exception as e:
            print(e)
            print(f"No directions between: {conflict_name} and {kk}")    


    basemaps['Google Satellite Hybrid'].add_to(map)
    basemaps['Google Maps'].add_to(map)
    # Add a layer control panel to the map.
    map.add_child(folium.LayerControl())    

    # Add fullscreen button
    plugins.Fullscreen().add_to(map)
    
    return map

In [ ]:
for kk, vv in conflicts.iterrows():
    name = vv['#name']
    print(f"Processing {name}")
    map = plot_conflict_focus(name,conflicts,camps)
    map = add_legend(map)
    map.save(f'conflict_focus_maps/{name}.html')

# Export closest routes

In [ ]:
from geojson import LineString, FeatureCollection, Feature
import json

In [ ]:
feats = []

for kk, vv in conflicts.iterrows():
    conflict = vv['#name']
    if conflict != 'Bashtanka':
        camp = closest[conflict]
        polyline_ = polyline.decode(all_directions[conflict][camp][0]['overview_polyline']['points'], geojson=True)
        feat = Feature(geometry=LineString(polyline_), properties={"conflict": conflict, "destination": camp})
        feats.append(feat)

feat_coll = FeatureCollection(feats)

In [ ]:
feat_coll

In [ ]:
with open('ukraine_routes.json','w') as f:
    f.write(json.dumps(feat_coll))